In [1]:
import mne
import pandas as pd

/var/folders/14/hz9wkgxn73dbs6fw9cz1vlsr0000gn/T/ipykernel_51271/170124415.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [81]:
# Path to your BDF file
# bdf_file_path = '/Users/jonathan/Documents/coding/alljoined/alljoined_preprocessing/raw/sub-04/subj04_session2.bdf' #works

subj_folder = 'sub-05'
bdf_file = 'subj05_session1'

bdf_file_path = f"/Users/jonathan/Documents/coding/alljoined/alljoined_preprocessing/raw/{subj_folder}/{bdf_file}.bdf"

# Read the BDF file
raw = mne.io.read_raw_bdf(bdf_file_path, preload=True)

Extracting EDF parameters from /Users/jonathan/Documents/coding/alljoined/alljoined_preprocessing/raw/sub-05/subj05_session1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1779199  =      0.000 ...  3474.998 secs...


In [3]:
print(raw.ch_names)

['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Status']


In [66]:
raw.plot(start=5, duration=5)

Using pyopengl with version 3.1.6


In [65]:
raw.copy().pick(picks="Status").plot(start=0, duration=6)

Using pyopengl with version 3.1.6


In [10]:
# Sampling rate
raw.info['sfreq']

512.0

In [61]:
events = mne.find_events(raw, stim_channel='Status', shortest_event=1)
# 65734 -> 198
# 65536 -> ?

Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
12719 events found on stim channel Status
Event IDs: [    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   151   152   153   154   155   156   157   158   159   160   161   162
   163   164   1

In [68]:
#(n_events, 3)
events.shape

(12719, 3)

In [92]:
# RTFM: https://mne.tools/stable/documentation/glossary.html#term-events
events[-10:]
# 198 = event_timestamp, , event_id


array([[1777956,       0,      76],
       [1778286,       0,     252],
       [1778344,       0,      16],
       [1778357,       0,     114],
       [1778713,       0,     252],
       [1778768,       0,      16],
       [1778779,       0,      44],
       [1779123,       0,     252],
       [1779181,       0,      16],
       [1779197,       0,     105]])

In [78]:
unique_event_ids = np.unique(events[:, 2])  # Extract unique event IDs
print(unique_event_ids)

'''
1. **198:** start
2. **199:** stop
3. **201:** train block
4. **202-217:** testing blocks
5. **START_RECORD = 198; % the trigger to start the EEG recording**
6. **STOP_RECORD = 199; % the trigger to stop the EEG recording**
7. **CORRECT_HIT = 254; % the trigger that records a correct hit to a catch stimulus**
8. **FALSE_ALARM = 253; % the trigger that records a false hit to an experimental stimulus**
9. **CORRECT_REJ = 252; % the trigger that records a correct no-key to an experiment stimulus**
10. **MISS = 251; % the trigger that records a false no-key to a catch stimulus**
'''

[    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   151   152   153   154   155   156   157   158   159   160   161   162
   163   164   165   166   201   202   203   204   205   206   207   208
   209   210   211   212   213   214   215   216   251   252   253   254
 65536 65734]


In [89]:
events_reversed = events[:, ::-1]  # Reverse the columns


mne.write_events(f"{bdf_file}.txt", events_reversed, overwrite=True)

Overwriting existing file.


/var/folders/14/hz9wkgxn73dbs6fw9cz1vlsr0000gn/T/ipykernel_51271/1147152723.py:2: RuntimeWarning: This filename (/Users/jonathan/Documents/coding/alljoined/alljoined_preprocessing/subj05_session1.txt) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(f"{bdf_file}.txt", events_reversed, overwrite=True)


In [157]:
def create_reaction_time_row(row, img_idx, reaction_time):
    new_row = row.copy()
    new_row.iloc[0] = str(img_idx)
    new_row['Reaction Time'] = reaction_time
    return new_row

In [164]:
i = 0
block_number = 1
oddball_offset = 150
error_count = 0
#TODO: read this from the saved raw file, bc we want to manually edit it and load in the edited version
raw_eeg_df = pd.DataFrame(events_reversed)

while i < raw_eeg_df.shape[0] - 2:
    event_code = raw_eeg_df.iloc[i, 0]
    event_timestamp = raw_eeg_df.iloc[i+2, 0]

    print(event_code)
    # Start of new block
    if event_code == 65734:
        block_number += 1
        i += 2

    # print(type(event_timestamp), type(event_code))

    # 254 - Oddball - Correct - hit 
    if event_code == 254 and event_timestamp == block_number+oddball_offset:
        reaction_time = raw_eeg_df.iloc[i+2, 2] - raw_eeg_df.iloc[i, 2]
        img_idx = ((block_number - 1) % 8) * 120 + int(raw_eeg_df.iloc[i+1, 0])
        # modified_row = create_reaction_time_row(raw_eeg_df.iloc[i], 20000, reaction_time)
    
    # # 251 - Oddball - Miss - no hit
    elif event_code == 251 and event_timestamp == block_number+oddball_offset:
        reaction_time = raw_eeg_df.iloc[i+2, 2] - raw_eeg_df.iloc[i, 2]
        img_idx = ((block_number - 1) % 8) * 120 + int(raw_eeg_df.iloc[i+1, 0])
        # modified_row = create_reaction_time_row(raw_eeg_df.iloc[i], 20000, reaction_time)
        
    # 252 - Correct - no hit  
    elif event_code == 252 and event_timestamp == block_number:
        reaction_time = raw_eeg_df.iloc[i+2, 2] - raw_eeg_df.iloc[i, 2]
        img_idx = ((block_number - 1) % 8) * 120 + int(raw_eeg_df.iloc[i+1, 0])
        modified_row = create_reaction_time_row(raw_eeg_df.iloc[i], img_idx, reaction_time)
        new_df = new_df._append(modified_row, ignore_index=True)
        
    # 253 - False hit 
    elif event_code == 253 and event_timestamp == block_number:
        reaction_time = raw_eeg_df.iloc[i+2, 2] - raw_eeg_df.iloc[i, 2]
        img_idx = ((block_number - 1) % 8) * 120 + int(raw_eeg_df.iloc[i+1, 0])
        modified_row = create_reaction_time_row(raw_eeg_df.iloc[i], img_idx, reaction_time)
        new_df = new_df._append(modified_row, ignore_index=True)

    else:
        if error_count < 10:
            error_count += 1
            print('Error: ', i, raw_eeg_df.iloc[i, 0], raw_eeg_df.iloc[i+2, 0])

    # Jump 3 rows for next trial

    i += 3

65734
Error:  2 1 252
1
Error:  5 1 252
151
Error:  8 151 254
1
Error:  11 1 252
1
Error:  14 1 252
1
Error:  17 1 252
1
Error:  20 1 252
1
Error:  23 1 252
151
Error:  26 151 254
1
Error:  29 1 252
1
1
1
1
1
1
1
1
151
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
151
1
151
1
1
1
151
1
151
1
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
1
151
1
1
1
1
1
1
1
151
1
1
1
1
1
151
1
1
1
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
1
151
1
1
151
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
151
1
1
1
1
151
1
65536
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
152
2
2
2
2
2
2
2
2
2
2
2
152
2
2
2
2
2
152
2
152
2
2
2
2
2
2
2
2
2
2
2
2
152
2
2
2
2
2
2
152
2
2
2
2
2
2
2
2
2
2
2
2
2
2
152
2
2
2
2
152
2
2
2
2
2
2
152
2
2
2
2
2
2
2
2
2
2
2
2
2
152
2
2
2
152

/var/folders/14/hz9wkgxn73dbs6fw9cz1vlsr0000gn/T/ipykernel_51271/3522097970.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_row.iloc[0] = str(img_idx)


NameError: name 'new_df' is not defined

: 

In [161]:
i

5057